In [3]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',shuffle=False)

twenty_train.data[0]

'F'

list

In [12]:
#efficient version
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect',CountVectorizer()),
                    ('tfidf',TfidfTransformer()),
                    ('clf',MultinomialNB())])



text_clf = text_clf.fit(twenty_train.data,twenty_train.target)

In [ ]:
import nltk
nltk.download()
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
stemmed_count_vect = StemmedCountVectorizer(stop_words='english')

In [43]:
text_clf_svm = Pipeline([('vect',CountVectorizer()),
                    ('tfidf',TfidfTransformer()),
                    ('clf_svm',SGDClassifier(loss="hinge",max_iter=1000,random_state=42))])

text_clf_svm = text_clf_svm.fit(twenty_train.data,twenty_train.target)

In [44]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test',shuffle=True)
predicted = text_clf_svm.predict(twenty_test.data)
np.mean(predicted == twenty_test.target)

0.8518321826872013

In [26]:
from sklearn.model_selection import GridSearchCV
"""
best params: default params
"""
params_svm= {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'clf_svm__penalty':['l1','l2'],
    'clf_svm__alpha':[.01,.001],
    'clf_svm__early_stopping':(True,False)
}

gs_clf = GridSearchCV(text_clf_svm,params_svm,n_jobs=-1)
gs_clf = gs_clf.fit(twenty_train.data,twenty_train.target)

In [27]:
twenty_test = fetch_20newsgroups(subset='test',shuffle=True)
predicted = gs_clf.predict(twenty_test.data)
np.mean(predicted == twenty_test.target)

0.8518321826872013